# Page Rank

What is the most important website on the internet? Who is the "key player" on a sports team? Which countries are the most central players in the world economy? There is no one correct answer to any of these questions, but there is a most profitable one. [PageRank](https://en.wikipedia.org/wiki/PageRank) is an algorithm for ranking individual elements of complex systems, invited by Sergey Brin and Larry Page. It was the first and most famous algorithm used by the Google Search engine, and it is fair to say that the internet as we know it today would not exist without PageRank. 

In this project, we will implement PageRank. There are many good ways to implement this algorithm, but in this assignment we will use our newfound skills with object-oriented programming and iterators. 

### How does it work?

For the purposes of this example, let's assume that we are talking about webpages. PageRank works by allowing a "random surfer" to move around webpages by following links. Each time the surfer lands on a page, it then looks for all the links on that page. It then picks one at random and follows it, thereby arriving at the next page, where the process repeats. Eventually, the surfer will visit all the pages one or more times. Pages that the surfer visits more frequently have higher *PageRank scores*. Because the surfer moves between linked pages, PageRank expresses an intuitive idea: **important pages are linked to other important pages.** [This diagram](https://en.wikipedia.org/wiki/PageRank#/media/File:PageRanks-Example.jpg) from Wikipedia gives a nice illustration. Note that more important webpages (higher PageRank) tend to be connected to other important webpages. 

<figure class="image" style="width:50%">
  <img src="https://upload.wikimedia.org/wikipedia/en/thumb/8/8b/PageRanks-Example.jpg/1920px-PageRanks-Example.jpg
" alt="A set of 11 circles, with arrows between them. Some of the circles are larger than others, reflecting their high PageRank score. Large circles tend to be linked to other large circles by arrows." width="300px">
  <figcaption><i>A schematic for PageRank. </i></figcaption>
</figure>


#### Hamilton Dataset

We will be using this data set that comes from the hit Broadway musical "Hamilton."

<figure class="image" style="width:25%">
  <img src="https://m.media-amazon.com/images/M/MV5BNjViNWRjYWEtZTI0NC00N2E3LTk0NGQtMjY4NTM3OGNkZjY0XkEyXkFqcGdeQXVyMjUxMTY3ODM@._V1_.jpg" alt="The logo of the musical Hamilton, showing a silhouette dressed in period custom standing on top of a five-pointed star." width="300px">
  <figcaption><i>The Hamilton data set</i></figcaption>
</figure>

The good folks at [The Hamilton Project](http://hamilton.newtfire.org/) analyzed the script for us, obtaining data on **who talks about whom** in each of the show's songs. When character A mentions character B, we'll think of this as a *link* from A to B, suggesting that B might be important. 

## Retrieving the Data

In [1]:
import urllib
import csv

def retrieve_data(url):
    """
    Retrieve a file from the specified url and save it in a local file 
    called data.csv. The intended values of url are: 
    
    https://philchodrow.github.io/PIC16A/homework/HW3-hamilton-data.csv
    """
    
    # grab the data and parse it
    filedata = urllib.request.urlopen(url) 
    to_write = filedata.read()
    
    # write to file
    with open("data.csv", "wb") as f:
        f.write(to_write)
        
def read_data(path):
    """
    read downloaded data from a .csv file, and return a list of tuples. 
    each tuple represents a link between states. 
    """
    with open(path, "r") as f:
        reader = csv.reader(f)
        return [(row[0], row[1]) for row in list(reader)]

## Grabbing the Data

In [2]:
url = "https://philchodrow.github.io/PIC16A/homework/HW3-hamilton-data.csv"

# Call your functions below
retrieve_data(url)          #retrieves the data from the url
data=read_data("data.csv")  #reads in the data from url and stores in variable data

## Examining the structure of the data

In [3]:
def describe(n):
    '''
    Describes the meaning of the nth row of the Hamilton data set
    Parameter n: the user-specified row to describe
    Return output: a string that describe the nth row
    '''
    N=str(n)         #converts to a string
    P=str(data[n])   #converts the nth row to a string
    n1=str(data[n][0]).capitalize()  #converts the first name in the nth row to a string and capitalizes it
    n2=str(data[n][1]).capitalize()  #converts the second name in the nth row to a string and capitalizes it
    
    #creates the string that describes the nth row 
    output="Element "+N+" of the Hamilton data set is "+P+". This means that "+n1+" mentions "+n2+ " in a song."
    
    return output    #returns the string

In [4]:
describe(5)          #tests out the describe function

"Element 5 of the Hamilton data set is ('burr', 'betsy'). This means that Burr mentions Betsy in a song."

## Converting the Data to a Dictionary

In [5]:
def data_to_dictionary(data):
    '''
    Converts the data into a dictionary where each key is a character in Hamilton and the value 
    corresponding to each key is a list of the characters to which that key links (including repeats)
    Parameter data: the user-supplied data set
    Return D: the dict with keys that are characters and values that are lists of characters 
    '''
    D={}   #create an empty dict
    
    #for loop to go through each line in the data set
    for line in data:
        #if statement to determine if the first item in the row (mentioner) is in the dict already
        if line[0] in D:
            #if it is, it will append the second item in the row (character mentioned) to the list
            D[line[0]].append(line[1])
        else:
        #creates a new entry in the dict, with mentioner as key and a list containing the character mentioned
            new={line[0]:[line[1]]}
            D.update(new)  #updates the dict with the new entry
             
    return D   #returns the dict

In [6]:
print(data_to_dictionary(data)) #prints the output for data_to_dictionary with argument data

{'burr': ['hamilton', 'weeks', 'madison', 'jay', 'theodosiaDaughter', 'betsy', 'theodosiaMother', 'hamilton', 'hamilton', 'hamilton', 'washington', 'hamilton', 'marthaWashington', 'schuylerSis', 'washington', 'burr', 'generalMontgomery', 'hamilton', 'philipS', 'peggy', 'angelica', 'eliza', 'hamilton', 'reynolds', 'hamilton', 'washington', 'hamilton', 'philipS', 'generalMercer', 'madison', 'jefferson', 'washington', 'hamilton', 'washington', 'jefferson', 'jefferson', 'madison', 'burr', 'hamilton', 'hamilton', 'jAdams', 'jefferson', 'hamilton', 'jefferson', 'burr', 'ness', 'hamilton', 'pendleton', 'angelica', 'eliza'], 'hamilton': ['burr', 'angelica', 'philipH', 'lafayette', 'eliza', 'laurens', 'mulligan', 'washington', 'eliza', 'lee', 'laurens', 'conway', 'hamilton', 'washington', 'lee', 'laurens', 'burr', 'washington', 'hamilton', 'burr', 'lee', 'burr', 'eliza', 'peggy', 'angelica', 'hamilton', 'laurens', 'mulligan', 'lafayette', 'burr', 'kingGeorge', 'burr', 'lafayette', 'laurens', 'b

## Defining a PR_DiGraph class

A **directed graph**, or DiGraph, is just a set of arrows ("edges") between objects ("nodes"). It is a natural way to represent data that represents one-way relationships, such as links from one webpage to another or mentions of one character by another. We already saw a directed graph above when we introduced the idea of PageRank. Here's a paired-down example. 

<figure class="image" style="width:50%">
  <img src="https://computersciencewiki.org/images/c/c6/Directed_graph.png" alt="Six circles, representing nodes, labeled A through F. There are directed arrows between certain pairs of nodes." width="300px">
  <figcaption><i>Example of a directed graph. </i></figcaption>
</figure>

In [7]:
class PR_DiGraph:
    '''
    A class which models a directed graph. 
    Each PR_DiGraph has data and iteration_limit specified by user.
    '''
    
    def __init__(self,data,iteration_limit):
        '''
        Contructs the necessary attributes for the PR_DiGraph object
        '''
        self.data=data                          #the data
        self.iteration_limit=iteration_limit    #the iteration_limit
        self.link_dict=data_to_dictionary(data) #output of data_to_dictionary applied to the argument data
        
    def linked_by(self,mentioner):
        '''
        Returns the list of characters mentioned by the mentioner from the Hamilton data set
        Parameter mentioner: the user-specified character mentioning other characters from Hamilton
        '''
        return self.link_dict[mentioner] #returns list of characters mentioned by the user-specified character
        
    def __iter__(self):
        '''
        Used to iterate over elements in object
        '''
        return(PR_Iterator(self)) 
        

In [8]:
#example test 
D = PR_DiGraph(data, iteration_limit = 10000)
D.linked_by('peggy')  #tests out linked_by member function from class PR_DiGraph

['peggy', 'schuylerSis']

## Implementing a PR_Iterator

We are going to use iteration to implement the PageRank algorithm. This  means we are going to imagine a surfer who is following the links in our data set. 

In [9]:
import random        #imports random module

class PR_Iterator:
    '''
    An iterator class for PR_DiGraph. 
    Goes by using follow_link with 85% probability and teleport() with 15% probability
    '''
    def __init__(self,graph):
        '''
        Contructs the necessary attributes for the PR_Iterator
        '''
        self.graph=graph               #the graph
        self.i=0                       #the counter i, starting at 0
        self.current_state="hamilton"  #the current_state set at "hamilton"
    
    def follow_link(self):
        '''
        Picks a random character mentioned by current character
        '''
        #picks a random character from list of characters mentioned by current character
        next_state=random.choice(self.graph.link_dict.get(self.current_state,self.current_state))
        #if statement to check if the character picked is the same one as previous
            
        if next_state != self.current_state:
            self.current_state=next_state #if not the same, set the next character as the current one
        else: 
            #if the character randomly picked is the same one as previous, go to teleport
            try:
                self.teleport()
            except:
                self.teleport()  
    
    def teleport(self):
        '''
        Picks a random character from the keys of link_dict
        '''
        #picks random character from the keys of link_dict
        next_state=random.choice(list(self.graph.link_dict.keys()))
        self.current_state=next_state   #set current_state to next_state
        
    def __next__(self):
        '''
        Gets the next iterator element
        '''
        #if statement to check if the iteration limit has been reached
        if self.i == self.graph.iteration_limit:
            raise StopIteration    #if it has, then stop the iteration
        
        #if statement to go to follow_link() with 85% probability
        if random.random() < 0.85:
            self.follow_link()
        else:  #goes to teleport() with 15% probability
            self.teleport()
        
        if len(self.current_state)==1:
            self.teleport()

        self.i+=1  #increase counter by one

        return self.current_state  #returns the current_state


In [10]:
# run the below
D = PR_DiGraph(data, iteration_limit = 5)
for char in D:
    print(char)

washington
hamilton
philipH
madison
hamilton


## Computing PageRank

In [11]:
#initializing a PR_DiGraph with iteration limit of 1,000,000
Dataset = PR_DiGraph(data, iteration_limit = 1000000)  
D={}   #creating an empty dict

#for loop to go through each item in Dataset
for mentioner in Dataset:
    #creates key for each mentioner, and adds one everytime it appears again throughout Dataset
    D[mentioner]=D.get(mentioner,0)+1
    
print(D) #prints the dict which logs how many times a given state appears when iterating through Dataset

{'eliza': 51527, 'jefferson': 72547, 'lafayette': 34369, 'burr': 99544, 'angelica': 47645, 'ensemble': 17262, 'kingGeorge': 28701, 'jAdams': 31091, 'mulligan': 21249, 'reynolds': 29276, 'hamilton': 166613, 'madison': 36714, 'laurens': 27565, 'washington': 92308, 'green': 3834, 'doctor': 17077, 'peggy': 20450, 'schuylerSis': 19031, 'women': 17065, 'company': 16899, 'sally': 2903, 'rochambeau': 4024, 'seabury': 16942, 'maria': 1798, 'lee': 33628, 'philipH': 26527, 'knox': 3941, 'men': 17014, 'marthaWashington': 1710, 'generalMercer': 1655, 'eacker': 6316, 'philipS': 7849, 'weeks': 1634, 'generalMontgomery': 1671, 'conway': 1780, 'pendleton': 1680, 'franklin': 1910, 'betsy': 1706, 'sAdams': 3361, 'kingLouis': 1670, 'ness': 1688, 'theodosiaMother': 1685, 'theodosiaDaughter': 1721, 'jay': 1696, 'admiralHowe': 725, 'paine': 1999}


## Displaying Results

Sorted in descending order according to the corresponding PageRank score

In [12]:
sorted(D.items(), key=lambda x: x[1], reverse=True)

[('hamilton', 166613),
 ('burr', 99544),
 ('washington', 92308),
 ('jefferson', 72547),
 ('eliza', 51527),
 ('angelica', 47645),
 ('madison', 36714),
 ('lafayette', 34369),
 ('lee', 33628),
 ('jAdams', 31091),
 ('reynolds', 29276),
 ('kingGeorge', 28701),
 ('laurens', 27565),
 ('philipH', 26527),
 ('mulligan', 21249),
 ('peggy', 20450),
 ('schuylerSis', 19031),
 ('ensemble', 17262),
 ('doctor', 17077),
 ('women', 17065),
 ('men', 17014),
 ('seabury', 16942),
 ('company', 16899),
 ('philipS', 7849),
 ('eacker', 6316),
 ('rochambeau', 4024),
 ('knox', 3941),
 ('green', 3834),
 ('sAdams', 3361),
 ('sally', 2903),
 ('paine', 1999),
 ('franklin', 1910),
 ('maria', 1798),
 ('conway', 1780),
 ('theodosiaDaughter', 1721),
 ('marthaWashington', 1710),
 ('betsy', 1706),
 ('jay', 1696),
 ('ness', 1688),
 ('theodosiaMother', 1685),
 ('pendleton', 1680),
 ('generalMontgomery', 1671),
 ('kingLouis', 1670),
 ('generalMercer', 1655),
 ('weeks', 1634),
 ('admiralHowe', 725)]

Looks like Hamilton is mentioned the most throughout the play, followed by Burr, Washington, then Jefferson.